In [8]:
#Inputs: Directory/path of raw manual csvs, directory/path of raw predicted csvs
#Outputs: Metrics csv with precision, recall, f1, iou, and relation categorization metrics
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon, GeometryCollection
from shapely.ops import unary_union
from shapely.validation import explain_validity
import os
import re
import chardet
from tqdm import tqdm
from sklearn.metrics import precision_recall_curve, average_precision_score
from skimage.measure import label, regionprops
from scipy.optimize import linear_sum_assignment
import numpy as np
import matplotlib.pyplot as plt
from shapely.validation import make_valid

def read_csv_to_polygons_old(data):
    rois = []
    grouped = data.groupby('Name').agg(list).reset_index()
    for _, row in grouped.iterrows():
        outline = list(zip(row['X'], row['Y']))
        if outline[0] != outline[-1]:
            outline.append(outline[0])
        polygon = Polygon(outline)
        rois.append(polygon)
    return rois

def read_csv_to_polygons(data, visualize_invalid=True):
    rois = []
    grouped = data.groupby('Name').agg(list).reset_index()

    for name, row in grouped.iterrows():
        outline = list(zip(row['X'], row['Y']))

        if len(outline) < 3:
            continue

        outline.append(outline[0])  # Close the loop

        if len(outline) < 4:
            continue

        polygon = Polygon(outline)

        if not polygon.is_valid:
            reason = explain_validity(polygon)
            print(f'Polygon for ROI "{name}" is invalid: {reason}.')
            valid_geom = make_valid(polygon)

            # Extract only Polygon parts
            if isinstance(valid_geom, Polygon):
                parts = [valid_geom]
            elif isinstance(valid_geom, MultiPolygon):
                parts = list(valid_geom.geoms)
            elif isinstance(valid_geom, GeometryCollection):
                parts = [g for g in valid_geom.geoms if isinstance(g, Polygon)]
            else:
                parts = []

            if parts:
                print(parts)
                # Find the polygon with the largest area
                largest_poly = max(parts, key=lambda p: p.area)
                rois.append(largest_poly)

            if visualize_invalid:
                # Plot original
                xs, ys = zip(*outline)
                plt.figure(figsize=(5, 5))
                plt.plot(xs, ys, marker='o', linestyle='-', label='Original (Invalid)')
                plt.title(f'Invalid ROI: {name}\nReason: {reason}')
                plt.gca().set_aspect('equal', adjustable='box')
                plt.grid(True)

                # Plot fixed parts
                valid_parts = []
                for i, poly in enumerate(parts):
                    x, y = poly.exterior.xy
                    
                    plt.plot(x, y, label=f'Valid Part {i+1}')

                plt.legend()
                plt.show()

            continue

        rois.append(polygon)

    return rois, len(grouped)

def list_of_intersections(l1, l2):
    inter = [(p1, p2) for p1 in l1 for p2 in l2 if p1.intersects(p2) and not p1.touches(p2)]
    return inter

def best_iou_list(manual_rois, predicted_rois):
    liou = []
    for man in manual_rois:
        max_iou = max([iou(man, pred) for pred in predicted_rois] or [0])
        liou.append(max_iou)
    return liou

def intersect(l1, l2, loi):
    inter = [p1.intersection(p2) for (p1, p2) in loi]
    return inter

def tot_iou(roi1, roi2, loi):
    union = unary_union(roi1 + roi2)
    inter = unary_union(intersect(roi1, roi2, loi))
    ua = union.area
    ia = inter.area
    if ua == 0:
        if ia == 0:
            return 1
        return 0
    return ia/ua

def iou(roi1, roi2):
    union = unary_union([roi1, roi2]).area
    inter = roi1.intersection(roi2).area
    return inter/union

def false_negatives(roi1, roi2):
    fn = 0
    tot = unary_union(roi2)
    for man in roi1:
        if not man.intersects(tot):
            fn += 1
    return fn

def false_positives(roi1, roi2):
    fp = 0
    tot = unary_union(roi1)
    for pred in roi2:
        if not pred.intersects(tot):
            fp += 1
    return fp

def true_positives_iou(roi1, roi2, loi, iou):
    tp = 0
    for (c1, c2) in loi:
        if iou(c1, c2) >= iou:
            tp += 1
    return tp

def relation_categorization_iou(l1, l2, loi, thresh):
    part = 0
    trans = 0
    endo = 0
    peri = 0
    tp = 0
    fp = false_positives(l1, l2)
    fn = false_negatives(l1, l2)
    for (c1, c2) in loi:
        if iou(c1, c2) > thresh:
            tp += 1
        elif c1.crosses(c2):
            trans += 1
        elif c1.within(c2):
            peri += 1
        elif c1.covers(c2):
            endo += 1
        elif c1.overlaps(c2):
            part += 1
    cat = {'True Positives': tp, 'False Positives': fp, 'False Negatives': fn, 'Partial Errors': part, 'Inner Errors': endo, 'Excess Errors': peri, 'Crossing Errors': trans}
    return cat
    
def recall(l1, l2, loi):
    if len(l1) == 0:
        if len(l2) != 0:
            return 0
        return None
    tot = unary_union(l1).area
    if tot == 0:
        if len(l2) == 0:
            return None
        return 0
    correct = unary_union(intersect(l1, l2, loi)).area
    return correct/tot

def precision(l1, l2, loi):
    if len(l2) == 0:
        if len(l1) != 0:
            return 0
        return None
    tot = unary_union(l2).area
    if tot == 0:
        if len(l1) == 0:
            return None
        return 0
    correct = unary_union(intersect(l1, l2, loi)).area
    return correct/tot

def f1_score(l1, l2, loi):
    if len(l1) == 0:
        if len(l2) != 0:
            return 0
        return None
    prec = precision(l1, l2, loi)
    rec = recall(l1, l2, loi)
    if prec+rec == 0:
        return 0
    return 2*prec*rec/(prec+rec)

def cell_prec(l1, l2, bound, liou):
    count = 0
    if len(l2) == 0:
        return None
    elif len(l1) == 0:
        return 0
    for i in liou:
        if i >= bound:
            count += 1
    return count/len(l2)

def cell_prec_50_95(l1, l2, liou):
    b = 0.5
    res = 0
    if len(l2) == 0:
        return None
    elif len(l1) == 0:
        return 0
    for i in range(10):
        res += cell_prec(l1, l2, b, liou)
        b += 0.05
    return res/10

def cell_rec(l1, l2, bound, liou):
    count = 0
    if len(l1) == 0:
        return None
    elif len(l2) == 0:
        return 0
    for i in liou:
        if i >= bound:
            count += 1
    return count/len(l1)

def cell_rec_50_95(l1, l2, liou):
    b = 0.5
    res = 0
    if len(l1) == 0:
        return None
    elif len(l2) == 0:
        return 0
    for i in range(10):
        res += cell_rec(l1, l2, b, liou)
        b += 0.05
    return res/10

def cell_f1(l1, l2, bound, liou):
    if len(l1) == 0 and len(l2) == 0:
        return None
    elif len(l1) == 0 or len(l2) == 0:
        return 0
    prec = cell_prec(l1, l2, bound, liou)
    rec = cell_rec(l1, l2, bound, liou)
    if prec == None or rec == None:
        return 0
    elif prec+rec == 0:
        return 0
    return 2*prec*rec/(prec+rec)

def cell_f1_50_95(l1, l2, liou):
    b = 0.5
    res = 0
    if len(l1) == 0 and len(l2) == 0:
        return None
    elif len(l1) == 0 or len(l2) == 0:
        return 0
    for i in range(10):
        res += cell_f1(l1, l2, b, liou)
        b += 0.05
    return res/10

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

def normalize_yolo(man_dir, pred_dir):
    for filename in os.listdir(man_dir):
        if filename not in os.listdir(pred_dir) and re.findall(r'\d+', filename) not in [re.findall(r'\d+', i) for i in os.listdir(pred_dir)]:
            df = pd.DataFrame(columns=['Name', 'X', 'Y'])
            df.to_csv(os.path.join(pred_dir, filename))

def list_mapping(i_dir, j_dir):
    aligned = []
    for i in i_dir:
        for j in j_dir:
            if i.split('.')[0] == j.split('.')[0]:
                aligned.append((i, j))
            elif re.findall(r'\d+', i) == re.findall(r'\d+', j):
                aligned.append((i, j))
    print(aligned)
    return aligned

def avg_col(d):
    sum_col = 0
    count = 0
    for i in d:
        if i is not None:
            sum_col += i
            count += 1
    if count == 0:
        return None
    return sum_col/count

def total_area(l1, l2):
    return unary_union(l1 + l2).area

def weighted_avg(values, weights, names):
    total = 0
    weight_sum = 0
    for v, w, name in zip(values, weights, names):
        if isinstance(v, (int, float)) and isinstance(w, (int, float)) and 'Statistics' not in name:
            total += v * w
            weight_sum += w
    return total / weight_sum if weight_sum != 0 else None

def object_avg(c, man, pred, n):
    tot_man = 0
    tot_pred = 0
    res = 0
    for i, j, m in zip(man, pred, n):
        if 'Statistics' not in m:
            tot_man += i
            tot_pred += j
    if tot_man+tot_pred == 0:
        return None
    for j, k, l, m in zip(c, man, pred, n):
        if j is not None and 'Statistics' not in m:
            if 'cell_prec' in m or 'Prec' in m:
                ratio = l/tot_pred
                res += j*ratio
            elif 'cell_rec' in m or 'Rec' in m:
                ratio = k/tot_man
                res += j*ratio
            else:
                ratio = (k+l)/(tot_man+tot_pred)
                res += j*ratio
    print(f'Result:', res)
    return res

def compute_ap(loi, num_gt, iou_thresh=0.5):
    matched_gt = set()
    matched_pred = set()
    tp = []
    fp = []

    # Compute IoU for all pairs in loi using your function
    ious_with_pairs = [
        (p1, p2, iou(p1, p2)) for (p1, p2) in loi
    ]
    ious_with_pairs.sort(key=lambda x: x[2], reverse=True)

    if len(ious_with_pairs) == 0:
        return 0
        
    for p1, p2, iou_val in ious_with_pairs:
        gt_id = id(p1)
        pred_id = id(p2)

        if iou_val >= iou_thresh and gt_id not in matched_gt and pred_id not in matched_pred:
            tp.append(1)
            fp.append(0)
            matched_gt.add(gt_id)
            matched_pred.add(pred_id)
        else:
            tp.append(0)
            fp.append(1)

    tp = np.cumsum(tp)
    fp = np.cumsum(fp)

    precisions = tp / (tp + fp + 1e-8)
    recalls = tp / num_gt

    precisions = np.concatenate([[1.0], precisions, [0.0]])
    recalls = np.concatenate([[0.0], recalls, [1.0]])

    for i in range(len(precisions) - 1, 0, -1):
        precisions[i - 1] = max(precisions[i - 1], precisions[i])

    ap = np.trapz(precisions, recalls)
    return ap

def compute_map(loi, num_gt, iou_thresh_start=0.5, iou_thresh_end=0.95, step=0.05):
    ap_values = []
    
    # Iterate through IoU thresholds from 0.5 to 0.95 (with a step of 0.05)
    for iou_thresh in np.arange(iou_thresh_start, iou_thresh_end + step, step):
        ap = compute_ap(loi, num_gt, iou_thresh)
        ap_values.append(ap)
    
    # Return the mean of the computed AP values for the IoU thresholds
    mAP_50_95 = np.mean(ap_values)
    return mAP_50_95

def optimized_model_eval(man_dir, pred_dir, thresh=0.5):
    performance = {
        'Name': [],
        'Total Area': [],
        'Total Manual Masks': [], 
        'Valid Manual Masks': [],
        'Total Predicted Masks': [], 
        'Valid Predicted Masks': [],
        'List of Intersections': [],
        'List of IoUs': [],
        'AP50': [],
        'mAP50-95': [],
        'cell_prec50': [],
        'cell_prec50-95': [],
        'cell_rec50': [],
        'cell_rec50-95': [],
        'cell_f1_50': [],
        'cell_f1_50-95': [],
        'Total IoU': [],
        'pix_prec': [],
        'pix_rec': [],
        'pix_f1': [],
        'List of Relations': []
    }
    
    man_files = [f for f in os.listdir(man_dir) if f.endswith('.csv')]
    pred_files = [f for f in os.listdir(pred_dir) if f.endswith('.csv')]
    print(man_files, pred_files)
    
    file_mapping = list_mapping(man_files, pred_files)
    print(file_mapping)

    sum_cats = {}
    for filename1, filename2 in tqdm(file_mapping, desc="Processing files"):
        man_path = os.path.join(man_dir, filename1)
        pred_path = os.path.join(pred_dir, filename2)
        
        if os.path.isfile(man_path) and os.path.isfile(pred_path):
            performance['Name'].append(filename1.replace('.csv', ''))
            
            # Read CSV files
            man_data = pd.read_csv(man_path, delimiter=',')
            pred_data = pd.read_csv(pred_path, delimiter=',')
            
            # Process data
            rois_man, orig_man = read_csv_to_polygons(man_data)
            rois_pred, orig_pred = read_csv_to_polygons(pred_data)
            loi = list_of_intersections(rois_man, rois_pred)
            
            # Calculate metrics
            tot_area = total_area(rois_man, rois_pred)
            iou_list = [iou(c1, c2) for c1, c2 in loi]

            # Assuming `relation_categorization_iou` returns a dictionary
            categories = relation_categorization_iou(rois_man, rois_pred, loi, thresh)
            
            # Iterate through the dictionary and sum the values based on keys
            for key, value in categories.items():
                if key in sum_cats:
                    sum_cats[key] += value
                else:
                    sum_cats[key] = value

            performance['AP50'].append(compute_ap(loi, len(rois_man)))
            performance['mAP50-95'].append(compute_map(loi, len(rois_man)))
            performance['Total Area'].append(tot_area)
            performance['Total Manual Masks'].append(orig_man)
            performance['Total Predicted Masks'].append(orig_pred)
            performance['Valid Manual Masks'].append(len(rois_man))
            performance['Valid Predicted Masks'].append(len(rois_pred))
            performance['List of Intersections'].append(loi)
            performance['List of IoUs'].append(iou_list)
            performance['Total IoU'].append(tot_iou(rois_man, rois_pred, loi))
            performance['cell_prec50'].append(cell_prec(rois_man, rois_pred, thresh, iou_list))
            performance['cell_prec50-95'].append(cell_prec_50_95(rois_man, rois_pred, iou_list))
            performance['cell_rec50'].append(cell_rec(rois_man, rois_pred, thresh, iou_list))
            performance['cell_rec50-95'].append(cell_rec_50_95(rois_man, rois_pred, iou_list))
            performance['cell_f1_50'].append(cell_f1(rois_man, rois_pred, thresh, iou_list))
            performance['cell_f1_50-95'].append(cell_f1_50_95(rois_man, rois_pred, iou_list))
            performance['pix_prec'].append(precision(rois_man, rois_pred, loi))
            performance['pix_rec'].append(recall(rois_man, rois_pred, loi))
            performance['pix_f1'].append(f1_score(rois_man, rois_pred, loi))
            performance['List of Relations'].append(categories)
    print(performance['AP50'])
    print(performance['mAP50-95'])
    
    # Add summary statistics
    def add_summary_statistic(stat_func):
        performance['Name'].append('Summary Statistics')
        performance['Total Area'].append(None)
        performance['Total Manual Masks'].append(None)  # Keep the original list intact
        performance['Total Predicted Masks'].append(None)
        performance['Valid Manual Masks'].append(None)  # Keep the original list intact
        performance['Valid Predicted Masks'].append(None)
        performance['List of Intersections'].append(None)
        performance['List of IoUs'].append(None)
        for key in performance:
            if key not in ['Name', 'List of Intersections', 'List of IoUs', 'List of Relations', 'Total Manual Masks', 'Total Predicted Masks', 'Valid Manual Masks', 'Valid Predicted Masks', 'Total Area']:
                performance[key].append(stat_func(performance[key]))  # Add summary statistic as a separate entry
        performance['List of Relations'].append(None)
    
    add_summary_statistic(avg_col)
    
    # Add weighted statistics
    def add_weighted_statistic(stat_func):
        performance['Name'].append('Pixel Weight Statistics')
        performance['Total Area'].append(None)
        performance['Total Manual Masks'].append(None)  # Keep the original list intact
        performance['Total Predicted Masks'].append(None)
        performance['Valid Manual Masks'].append(None)  # Keep the original list intact
        performance['Valid Predicted Masks'].append(None)
        performance['List of Intersections'].append(None)
        performance['List of IoUs'].append(None)
        for key in performance:
            if key not in ['Name', 'List of Intersections', 'List of IoUs', 'List of Relations', 'Total Manual Masks', 'Total Predicted Masks', 'Valid Manual Masks', 'Valid Predicted Masks', 'Total Area']:
                performance[key].append(stat_func(performance[key], performance['Total Area'], performance['Name']))  # Add weighted statistic as a separate entry
        performance['List of Relations'].append(None)
    
    add_weighted_statistic(weighted_avg)

    # Add cell statistics
    def add_cell_weight_statistic(stat_func, sum_cats):
        performance['Name'].append('Cell Weight Statistics')
        performance['Total Area'].append(None)
        performance['Total Manual Masks'].append(None)  # Keep the original list intact
        performance['Total Predicted Masks'].append(None)
        performance['Valid Manual Masks'].append(None)  # Keep the original list intact
        performance['Valid Predicted Masks'].append(None)
        performance['List of Intersections'].append(None)
        performance['List of IoUs'].append(None)
        for key in performance:
            if key not in ['Name', 'List of Intersections', 'List of IoUs', 'List of Relations', 'Total Manual Masks', 'Total Predicted Masks', 'Valid Manual Masks', 'Valid Predicted Masks', 'Total Area', 'cell_f1_50', 'cell_f1_50-95']:
                performance[key].append(stat_func(performance[key], performance['Total Manual Masks'], performance['Total Predicted Masks'], performance['Name']))  # Add cell-weighted statistic as a separate entry
        for key in ['cell_f1_50', 'cell_f1_50-95']:
            prec = performance[key.replace('f1_', 'prec')][-1]
            rec = performance[key.replace('f1_', 'rec')][-1]
            if prec == None and rec == None:
                f1 = None
            elif prec == None or rec == None:
                f1 =  0
            elif prec+rec == 0:
                f1 = 0
            else:
                f1 = 2*prec*rec/(prec+rec)
            performance[key].append(f1)
        performance['List of Relations'].append(sum_cats)
    
    add_cell_weight_statistic(object_avg, sum_cats)
    
    return performance


#Evaluation for single-file
def optimized_model_eval_files(man_path, pred_path, thresh=0.5):
    performance = {
        'Name': [],
        'Total Area': [],
        'Total Manual Masks': [], 
        'Valid Manual Masks': [],
        'Total Predicted Masks': [], 
        'Valid Predicted Masks': [],
        'List of Intersections': [],
        'List of IoUs': [],
        'AP50': [],
        'mAP50-95': [],
        'cell_prec50': [],
        'cell_prec50-95': [],
        'cell_rec50': [],
        'cell_rec50-95': [],
        'cell_f1_50': [],
        'cell_f1_50-95': [],
        'Total IoU': [],
        'pix_prec': [],
        'pix_rec': [],
        'pix_f1': [],
        'List of Relations': []
    }
    
    performance['Name'].append(pred_path.replace('.csv', ''))
            
    # Read CSV files
    man_data = pd.read_csv(man_path, delimiter=',')
    print(man_data)
    pred_data = pd.read_csv(pred_path, delimiter=',')
            
    # Process data
    print('Manual conversion')
    rois_man, orig_man = read_csv_to_polygons(man_data, False)
    print(len(rois_man))
    print(orig_man)
    print('Prediction conversion')
    rois_pred, orig_pred = read_csv_to_polygons(pred_data, False)
    print(len(rois_pred))
    print(orig_pred)
    loi = list_of_intersections(rois_man, rois_pred)
            
    # Calculate metrics
    tot_area = total_area(rois_man, rois_pred)
    iou_list = [iou(c1, c2) for c1, c2 in loi]

    performance['AP50'].append(compute_ap(loi, len(rois_man)))
    performance['mAP50-95'].append(compute_map(loi, len(rois_man)))
    performance['Total Area'].append(tot_area)
    performance['Total Manual Masks'].append(orig_man)
    performance['Total Predicted Masks'].append(orig_pred)
    performance['Valid Manual Masks'].append(len(rois_man))
    performance['Valid Predicted Masks'].append(len(rois_pred))
    performance['List of Intersections'].append(loi)
    performance['List of IoUs'].append(iou_list)
    performance['Total IoU'].append(tot_iou(rois_man, rois_pred, loi))
    performance['cell_prec50'].append(cell_prec(rois_man, rois_pred, thresh, iou_list))
    performance['cell_prec50-95'].append(cell_prec_50_95(rois_man, rois_pred, iou_list))
    performance['cell_rec50'].append(cell_rec(rois_man, rois_pred, thresh, iou_list))
    performance['cell_rec50-95'].append(cell_rec_50_95(rois_man, rois_pred, iou_list))
    performance['cell_f1_50'].append(cell_f1(rois_man, rois_pred, thresh, iou_list))
    performance['cell_f1_50-95'].append(cell_f1_50_95(rois_man, rois_pred, iou_list))
    performance['pix_prec'].append(precision(rois_man, rois_pred, loi))
    performance['pix_rec'].append(recall(rois_man, rois_pred, loi))
    performance['pix_f1'].append(f1_score(rois_man, rois_pred, loi))
    performance['List of Relations'].append(relation_categorization_iou(rois_man, rois_pred, loi, thresh))
    print(performance['cell_f1_50'])
    print(performance['cell_f1_50-95'])
    return performance

def short_optimized_model_eval_files(man_path, pred_path, thresh = 0.5):
    performance = {
        'Name': [],
        'Total Area': [],
        'Total Manual Masks': [], 
        'Valid Manual Masks': [],
        'Total Predicted Masks': [], 
        'Valid Predicted Masks': [],
        'List of Intersections': [],
        'List of IoUs': [],
        'AP50': [],
        'mAP50-95': [],
        'cell_prec50': [],
        'cell_prec50-95': [],
        'cell_rec50': [],
        'cell_rec50-95': [],
        'cell_f1_50': [],
        'cell_f1_50-95': [],
        'Total IoU': [],
        'pix_prec': [],
        'pix_rec': [],
        'pix_f1': []
    }
    
    performance['Name'].append(pred_path.replace('.csv', ''))
            
    # Read CSV files
    man_data = pd.read_csv(man_path, delimiter=',')
    pred_data = pd.read_csv(pred_path, delimiter=',')
            
    # Process data
    rois_man, orig_man = read_csv_to_polygons(man_data)
    rois_pred, orig_pred = read_csv_to_polygons(pred_data)
    loi = list_of_intersections(rois_man, rois_pred)
            
    # Calculate metrics
    tot_area = total_area(rois_man, rois_pred)
    iou_list = [iou(c1, c2) for c1, c2 in loi]

    performance['AP50'].append(compute_ap(loi, len(rois_man)))
    performance['mAP50-95'].append(compute_map(loi, len(rois_man)))
    performance['Total Area'].append(tot_area)
    performance['Total Manual Masks'].append(orig_man)
    performance['Total Predicted Masks'].append(orig_pred)
    performance['Valid Manual Masks'].append(len(rois_man))
    performance['Valid Predicted Masks'].append(len(rois_pred))
    performance['List of Intersections'].append(loi)
    performance['List of IoUs'].append(iou_list)
    performance['Total IoU'].append(tot_iou(rois_man, rois_pred, loi))
    performance['cell_prec50'].append(cell_prec(rois_man, rois_pred, thresh, iou_list))
    performance['cell_prec50-95'].append(cell_prec_50_95(rois_man, rois_pred, iou_list))
    performance['cell_rec50'].append(cell_rec(rois_man, rois_pred, thresh, iou_list))
    performance['cell_rec50-95'].append(cell_rec_50_95(rois_man, rois_pred, iou_list))
    performance['cell_f1_50'].append(cell_f1(rois_man, rois_pred, thresh, iou_list))
    performance['cell_f1_50-95'].append(cell_f1_50_95(rois_man, rois_pred, iou_list))
    performance['pix_prec'].append(precision(rois_man, rois_pred, loi))
    performance['pix_rec'].append(recall(rois_man, rois_pred, loi))
    performance['pix_f1'].append(f1_score(rois_man, rois_pred, loi))
    return performance

#FOLDER BASED EVALUATION
man_dir = '/media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Cellpose/Batch2/DRG123/UnremovedTranscriptsTxt/Test'
pred_dir = '/media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Yolo/test_predictions/csv/'
out_path = ''

#normalize_yolo(man_dir, pred_dir)
#performance = optimized_model_eval(man_dir, pred_dir)

#df_csv = pd.DataFrame.from_dict(performance)
#df_csv.to_csv(out_path)

#FILE BASED EVALUATION
#Input: path to manual annotations (csv), path to predicted annotations (csv), output path (csv)
#Output: csv of relevant evaluation metrics (IoU, precision, recall, f1-score, etc.)
man_path = '/media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Xenium_segementation/ManuscriptProject/Scaled_DRG_Manual_Annotations/tg1_manual_scaled.csv'
pred_path = '/media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Cellpose/DRG123_Merged_New/DRG Merged Images/Eval/tg1.csv'
out_path = '/media/yhs/5596744f-db7c-442f-9235-d0c9d50c0a6b/Cellpose/DRG123_Merged_New/DRG Merged Images/Eval/tg1_eval.csv'

#performance = optimized_model_eval_files(man_path, pred_path)

#df_csv = pd.DataFrame.from_dict(performance)
#df_csv.to_csv(out_path)
print('Ready')F

Ready
